# Showing pitfalls in CV with Bulk Modulus data
### Julien Brenneck, July 2018
Specifically showing how the CV score in model selection can be overfit.

In [69]:
# Load sklearn modules
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.base import TransformerMixin, BaseEstimator

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR, LinearSVR
from sklearn.kernel_ridge import KernelRidge
from sklearn.neural_network import MLPRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from sklearn.neighbors import KNeighborsRegressor

from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RepeatedKFold, cross_val_score, cross_val_predict, train_test_split, GridSearchCV, RandomizedSearchCV, LeaveOneOut, KFold

import numpy as np
from pandas import DataFrame

# Load featurizers and feature utility functions
from matminer.featurizers.composition import ElementProperty
from matminer.featurizers.composition import OxidationStates
from matminer.featurizers.structure import DensityFeatures

from matminer.utils.conversions import composition_to_oxidcomposition
from matminer.utils.conversions import str_to_composition
from matminer.utils.pipeline import DropExcluded, ItemSelector

from matminer.datasets.dataframe_loader import load_elastic_tensor

In [3]:
# gives progress bar for loops, fall back to default range
try:
    from tqdm import tnrange, tqdm_notebook, trange
except ImportError:
    def tnrange(n, desc=''):
        return range(n)
    trange=range

In [4]:
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

import matplotlib as mpl
import matplotlib.pyplot as plt
plt.style.use('seaborn')

import seaborn
seaborn.set_color_codes()

## Load bulk modulus / elastic tensor data from matminer.
See pipeline notebook in matminer examples.

In [5]:
df = load_elastic_tensor()  # loads dataset in a pandas DataFrame 
unwanted_columns = ["volume", "nsites", "compliance_tensor", "elastic_tensor", 
                    "elastic_tensor_original", "K_Voigt", "G_Voigt", "K_Reuss", "G_Reuss"]
df = df.drop(unwanted_columns, axis=1)

In [6]:
# seperate out values to be estimated
y = df['K_VRH'].values

In [7]:
df["composition"] = df["formula"].transform(str_to_composition)
ep_feat = ElementProperty.from_preset(preset_name="magpie")
df = ep_feat.featurize_dataframe(df, col_id="composition")  # input the "composition" column to the featurizer

# can't use this util method in pipeline
df["composition_oxid"] = composition_to_oxidcomposition(df["composition"])

In [8]:
# columns to remove before regression
excluded = ["G_VRH", "K_VRH", "elastic_anisotropy", "formula", "material_id", 
            "poisson_ratio", "structure", "composition", "composition_oxid"]

# featurization transformations
featurizer = FeatureUnion(
    transformer_list=[
        ('drop', DropExcluded(excluded)),
        ('density', Pipeline([
            ('select', ItemSelector("structure")),
            ('density_feat', DensityFeatures()),
        ])),
        ('oxidation', Pipeline([
            ('select', ItemSelector("composition_oxid")),
            ('oxidation_feat', OxidationStates()),
        ])),
    ]
)

In [9]:
X = featurizer.transform(df)

## Fit KRR
Select specific splits to show the effect of overfitting in model selection.

In [189]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=21221) #1211 #21 #31 #0 #1009 #21221
np.random.seed(10)
model = KernelRidge(kernel='poly', degree=2)
param_grid = [{'alpha': np.arange(10,1000,10), 'gamma': 10.0**np.arange(-6.0,-2.0)}]
gs = GridSearchCV(model,
                  param_grid,
                  n_jobs=-1,
                  cv=KFold(10, random_state=100),
                  verbose=True,
                  return_train_score=True,
                  scoring='neg_mean_squared_error')
gs.fit(X_train, y_train)
print("CV RMSE:", np.sqrt(-gs.best_score_))
print(gs.best_params_)
print("Test RMSE:", np.sqrt(-gs.score(X_test, y_test)))
print("Train RMSE:", np.sqrt(-gs.score(X_train, y_train)))
grid = GridSearchCV(model, param_grid, cv=KFold(5), scoring='neg_mean_squared_error')
pipe = make_pipeline(grid)
s = cross_val_score(pipe, X_train, y_train, cv=KFold(5), n_jobs=-1, verbose=True, scoring='neg_mean_squared_error')
print("Nested CV RMSE:", np.sqrt(np.mean(-s)))

Fitting 10 folds for each of 396 candidates, totalling 3960 fits


[Parallel(n_jobs=-1)]: Done  72 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 904 tasks      | elapsed:    2.0s
[Parallel(n_jobs=-1)]: Done 2304 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done 3960 out of 3960 | elapsed:    7.0s finished


CV RMSE: 19.376987085725023
{'alpha': 990, 'gamma': 0.001}
Test RMSE: 25.98573500510255
Train RMSE: 11.594467339121131
Nested CV RMSE: 20.6159966267333


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   12.1s finished


## Fit GBT

In [194]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=21221) #1001 #12 #16 #21221 #1111
np.random.seed(10)
model = GradientBoostingRegressor()
param_grid = [{'max_depth': [1,2], 'n_estimators': [10000]}]
gs = GridSearchCV(model,
                  param_grid,
                  n_jobs=-1,
                  cv=KFold(10, random_state=100),
                  verbose=True,
                  return_train_score=True,
                  scoring='neg_mean_squared_error')
gs.fit(X_train, y_train)
print("CV RMSE:", np.sqrt(-gs.best_score_))
print(gs.best_params_)
print("Test RMSE:", np.sqrt(-gs.score(X_test, y_test)))
print("Train RMSE:", np.sqrt(-gs.score(X_train, y_train)))

Fitting 10 folds for each of 2 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  1.7min finished


CV RMSE: 14.514982117572337
{'max_depth': 1, 'n_estimators': 100000}
Test RMSE: 20.869513321088625
Train RMSE: 0.28845068533474844


In [193]:
np.random.seed(10)
model = GradientBoostingRegressor()
param_grid = [{'max_depth': [1,2], 'n_estimators': [10000]}]
grid = GridSearchCV(model, param_grid, cv=KFold(5), n_jobs=1, scoring='neg_mean_squared_error')
pipe = make_pipeline(grid)
s = cross_val_score(pipe, X_train, y_train, cv=KFold(5), n_jobs=-1, verbose=True, scoring='neg_mean_squared_error')
print("Nested CV RMSE:", np.sqrt(np.mean(-s)))

Nested CV RMSE: 16.380843360225082


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  1.8min finished


## Fit GP

In [174]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=12) #1001 #12 #16 #21
np.random.seed(10)
model = GaussianProcessRegressor()
param_grid = [{'kernel':[Matern(nu=0.25)]}]
gs = GridSearchCV(model,
                  param_grid,
                  n_jobs=-1,
                  cv=KFold(10, random_state=100),
                  verbose=True,
                  return_train_score=True,
                  scoring='neg_mean_squared_error')
gs.fit(X_train, y_train)
print("CV RMSE:", np.sqrt(-gs.best_score_))
print(gs.best_params_)
print("Test RMSE:", np.sqrt(-gs.score(X_test, y_test)))
print("Train RMSE:", np.sqrt(-gs.score(X_train, y_train)))
grid = GridSearchCV(model, param_grid, cv=KFold(5), scoring='neg_mean_squared_error')
pipe = make_pipeline(grid)
s = cross_val_score(pipe, X_train, y_train, cv=KFold(5), n_jobs=-1, verbose=True, scoring='neg_mean_squared_error')
print("Nested CV RMSE:", np.sqrt(np.mean(-s)))

Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Done   4 out of  10 | elapsed:    5.5s remaining:    8.2s
/global/homes/j/jbrennec/rsconda/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning:

fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([2.60188]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 29, 'nit': 2, 'warnflag': 2}

[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   14.3s finished


CV RMSE: 32.95206389775767
{'kernel': Matern(length_scale=1, nu=0.25)}
Test RMSE: 35.68066267756345
Train RMSE: 8.692384192839179e-07
Nested CV RMSE: 33.301186322394344


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   21.7s finished


## Fit MLP (NN)

In [140]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=1001) #1001 #12 #16 #21
np.random.seed(100)
model = MLPRegressor()
param_grid = [{'alpha': 10**np.arange(-8.0,-1.0)}]
gs = GridSearchCV(model,
                  param_grid,
                  n_jobs=-1,
                  cv=KFold(10, random_state=100),
                  verbose=True,
                  return_train_score=True,
                  scoring='neg_mean_squared_error')
gs.fit(X_train, y_train)
print("CV RMSE:", np.sqrt(-gs.best_score_))
print(gs.best_params_)
print("Test RMSE:", np.sqrt(-gs.score(X_test, y_test)))
print("Train RMSE:", np.sqrt(-gs.score(X_train, y_train)))
grid = GridSearchCV(model, param_grid, cv=KFold(5), scoring='neg_mean_squared_error')
pipe = make_pipeline(grid)
s = cross_val_score(pipe, X_train, y_train, cv=KFold(5), n_jobs=-1, verbose=True, scoring='neg_mean_squared_error')
print("Nested CV RMSE:", np.sqrt(np.mean(-s)))

Fitting 10 folds for each of 7 candidates, totalling 70 fits


[Parallel(n_jobs=-1)]: Done  14 out of  70 | elapsed:    1.5s remaining:    6.0s
[Parallel(n_jobs=-1)]: Done  70 out of  70 | elapsed:    2.8s finished


CV RMSE: 31.412817927918073
{'alpha': 1e-05}
Test RMSE: 34.56220762808273
Train RMSE: 26.19117625094989
Nested CV RMSE: 74.9726875815911


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    4.8s finished


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=1001) #1001 #12 #16 #21
np.random.seed(100)
model = MLPRegressor()
param_grid = [{'alpha': 10**np.arange(-8.0,-1.0)}]
gs = GridSearchCV(model,
                  param_grid,
                  n_jobs=-1,
                  cv=KFold(10, random_state=100),
                  verbose=True,
                  return_train_score=True,
                  scoring='neg_mean_squared_error')
gs.fit(X_train, y_train)
print("CV RMSE:", np.sqrt(-gs.best_score_))
print(gs.best_params_)
print("Test RMSE:", np.sqrt(-gs.score(X_test, y_test)))
print("Train RMSE:", np.sqrt(-gs.score(X_train, y_train)))
grid = GridSearchCV(model, param_grid, cv=KFold(5), scoring='neg_mean_squared_error')
pipe = make_pipeline(grid)
s = cross_val_score(pipe, X_train, y_train, cv=KFold(5), n_jobs=-1, verbose=True, scoring='neg_mean_squared_error')
print("Nested CV RMSE:", np.sqrt(np.mean(-s)))

In [ ]:
from psutil import virtual_memory
virtual_memory()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=111) #10

## Fit RF

In [ ]:
np.random.seed(10)
model = RandomForestRegressor()
param_grid = [{'n_estimators': np.arange(25, 500, 25)}]
gs = GridSearchCV(model,
                  param_grid,
                  n_jobs=-1,
                  cv=KFold(10, random_state=100),
                  verbose=True,
                  return_train_score=True,
                  scoring='neg_mean_squared_error')
gs.fit(X_train, y_train)
print("CV RMSE:", np.sqrt(-gs.best_score_))
# print(gs.best_params_)
print("Test RMSE:", np.sqrt(-gs.score(X_test, y_test)))
print("Train RMSE:", np.sqrt(-gs.score(X_train, y_train)))

In [ ]:
np.random.seed(100)
model = RandomForestRegressor()
param_grid = [{'n_estimators': np.arange(500, 550, 50)}]
grid = GridSearchCV(model, param_grid, cv=KFold(5), scoring='neg_mean_squared_error')
pipe = make_pipeline(grid)
s = cross_val_score(pipe, X_test, y_test, cv=KFold(5), n_jobs=-1, verbose=True, scoring='neg_mean_squared_error')
print("Nested CV RMSE:", np.sqrt(np.mean(-s)))

In [ ]:
x_scale = list(x['n_estimators'] for x in gs.cv_results_['params'])
plt.plot(x_scale, np.sqrt(-gs.cv_results_['mean_test_score']))
# plt.plot(x_scale, np.sqrt(-gs.cv_results_['mean_train_score']))
plt.show()

### Code to try lots of different splits and look at results in aggregate
next time use KRR for this as it runs quite fast

In [ ]:
# NTRIALS = 25
# KCV_RMSE = np.zeros(NTRIALS)
# TEST_RMSE = np.zeros(NTRIALS)
# TRAIN_RMSE = np.zeros(NTRIALS)
# NESTCV_RMSE = np.zeros(NTRIALS)

# model = RandomForestRegressor()
# param_grid = [{'n_estimators': np.arange(50, 300, 50)}]
# gs = GridSearchCV(model, param_grid, n_jobs=-1, cv=KFold(25),
#                   return_train_score=True, scoring='neg_mean_squared_error')
# pipe = make_pipeline(GridSearchCV(model, param_grid, cv=KFold(5), scoring='neg_mean_squared_error'))
# for i in trange(NTRIALS):
#     _X_train, _X_test, _y_train, _y_test = train_test_split(X, y, test_size=0.5)
#     gs.fit(_X_train, _y_train)
#     _s = cross_val_score(pipe, _X_train, _y_train, cv=KFold(10), n_jobs=-1, scoring='neg_mean_squared_error')
#     KCV_RMSE[i] = np.sqrt(-gs.best_score_)
#     TEST_RMSE[i] = np.sqrt(-gs.score(_X_test, _y_test))
#     TRAIN_RMSE[i] = np.sqrt(-gs.score(_X_train, _y_train))
#     NESTCV_RMSE[i] = np.sqrt(np.mean(-_s))

In [ ]:
# plt.figure(figsize=(5,5))
# x_s = np.arange(KCV_RMSE.size)
# plt.hlines([np.mean(KCV_RMSE), np.mean(TEST_RMSE), np.mean(NESTCV_RMSE)], linestyles='--', xmin=0,xmax=KCV_RMSE.size-1, colors=['b', 'g', 'r'], zorder=0)
# plt.scatter(x_s, KCV_RMSE, c='b', label='CV')
# plt.scatter(x_s, TEST_RMSE, c='g', label='Test')
# plt.scatter(x_s, NESTCV_RMSE, c='r', label='Nest')
# # plt.scatter(x_s, TRAIN_RMSE, c='k', label='Train')
# plt.xlabel("Trial")
# plt.ylabel("RMSE")
# plt.title("Overfitting CV Score in Model Selection")
# plt.legend()
# # plt.savefig('overfit-cv-scores.pdf')
# plt.show()